In [1]:
spark

In [2]:
# import modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
# need export ARROW_PRE_0_15_IPC_FORMAT=1 in .bashrc
import time
from collections import defaultdict
from datetime import datetime, date
import databricks.koalas as ks
# Application created time
data=[["1"]]
Appcreatetime=spark.createDataFrame(data,["id"])
Appcreatetime.withColumn("App_created_at",fn.current_timestamp()) \
    .show(truncate=False)

# Figures inline and set visualization style
%matplotlib inline
sns.set()
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
# except local mode
ks.set_option("compute.default_index_type", "distributed")

# Check spark app name
spark.sparkContext.appName

+---+-----------------------+
|id |App_created_at         |
+---+-----------------------+
|1  |2021-11-03 15:26:17.281|
+---+-----------------------+



'spark-15'

In [3]:
import datetime, string,time
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn

In [4]:
# need export ARROW_PRE_0_15_IPC_FORMAT=1 in .bashrc
import databricks.koalas as ks
ks.set_option("compute.default_index_type", "distributed")
ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.max_rows', 1000000)

In [5]:
start = time.time()

In [6]:
import time
from collections import defaultdict
from datetime import datetime, date

transactions_file = "./test/transactions.csv"
offers_file = "./test/offers.csv"
reduced_file = "./test/reduced.csv" 
def reduce_data(offers_file, transactions_file, reduced_file): #篩選出catergory或company之後會有優惠的購物紀錄
	start = datetime.now()
	#把offer檔案中的catergory.company找出來做成字典(把有提供優惠的公司跟產品找出)
	offers_cat = {}
	offers_co = {}
	for e, line in enumerate( open(offers_file) ):
		offers_cat[ line.split(",")[1] ] = 1
		offers_co[ line.split(",")[3] ] = 1
	#open output file
	with open(reduced_file, "wb") as outfile:
		#go through transactions file and reduce
		reduced = 0
		for e, line in enumerate( open(transactions_file) ):
			if e == 0:
				outfile.write( line.encode() ) #print header
			else:
				#只寫入catergory跟company有出現在offers字典的
				if line.split(",")[3] in offers_cat :
					outfile.write( line.encode() )
					reduced += 1
			#progress  
			if e % 50000000 == 0:
				print(e, reduced, datetime.now() - start)
	print(e, reduced, datetime.now() - start)
if __name__ == '__main__':
	reduce_data(offers_file, transactions_file, reduced_file)

0 0 0:00:00.038768
50000000 2362442 0:00:40.099291
100000000 4731553 0:01:16.694310
150000000 6852008 0:01:51.120082
200000000 8899249 0:02:24.489032
250000000 10995198 0:02:59.562346
300000000 13074178 0:03:35.213400
349655789 15349956 0:04:09.717359


In [7]:
# Read CVS file
fi = ks.read_csv('/data/reduced_test.csv')

In [8]:
fi.head()

,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount
8589934592,401466568,101,17,1726,101920010,10882,2013-02-23,35.00,CT,1,2.00
8589934593,401466568,101,56,5616,104610040,15889,2013-02-23,6.67,OZ,1,3.29
8589934594,401466568,101,56,5616,1068826767,81691,2013-03-02,8.00,OZ,1,2.89
8589934595,401466568,101,35,3509,103320030,875,2013-03-23,50.00,OZ,4,10.70
8589934596,401466568,101,56,5616,104610040,15889,2013-03-23,7.00,OZ,1,3.00


In [9]:
fi.dtypes

id                    int64
chain                 int32
dept                  int32
category              int32
company               int64
brand                 int32
date                 object
productsize         float64
productmeasure       object
purchasequantity      int32
purchaseamount      float64
dtype: object

In [10]:
df=fi.drop_duplicates(['date','id'],keep='first').groupby(['date','category']).count().sort_index()
df

id  chain  dept  company  brand  productsize  productmeasure  purchasequantity  purchaseamount
date       category                                                                                                  
2012-03-02 706        289    289   289      289    289          289             289               289             289
           799         12     12    12       12     12           12              12                12              12
           1703       670    670   670      670    670          670             670               670             670
           1726       313    313   313      313    313          313             313               313             313
           2119      2134   2134  2134     2134   2134         2134            2134              2134            2134
           2202        24     24    24       24     24           24              24                24              24
           3203       368    368   368      368    368          368             368               368             368
           3504      1006   1006  1006     1006   1006         1006            1006              1006            1006
           3509      1273   1273  1273     1273   1273         1273            1273              1273            1273
           4401       198    198   198      198    198          198             198               198             198
           4517       146    146   146      146    146          146             146               146             146
           5122       327    327   327      327    327          327             327               327             327
           5558      1153   1153  1153     1153   1153         1153            1153              1153            1153
           5616      1388   1388  1388     1388   1388         1388            1388              1388            1388
           5619       219    219   219      219    219          219             219               219             219
           5824      1398   1398  1398     1398   1398         1398            1398              1398            1398
           6202       198    198   198      198    198          198             198               198             198
           7205      1527   1527  1527     1527   1527         1527            1527              1527            1527
           9115       102    102   102      102    102          102             102               102             102
           9909      5248   5248  5248     5248   5248         5248            5248              5248            5248
2012-03-03 706        305    305   305      305    305          305             305               305             305
           799         16     16    16       16     16           16              16                16              16
           1703       836    836   836      836    836          836             836               836             836
           1726       421    421   421      421    421          421             421               421             421
           2119      2545   2545  2545     2545   2545         2545            2545              2545            2545
           2202        33     33    33       33     33           33              33                33              33
           3203       425    425   425      425    425          425             425               425             425
           3504      1205   1205  1205     1205   1205         1205            1205              1205            1205
           3509      1471   1471  1471     1471   1471         1471            1471              1471            1471
           4401       204    204   204      204    204          204             204               204             204
           4517       202    202   202      202    202          202             202               202             202
           5122       364    364   364      364    364          364             364               364             364
           

In [11]:
aggregated = fi.groupby(['date','category']).agg({'purchaseamount': 'sum'})
aggregated.sort_index()

purchaseamount
date       category                
2012-03-02 706              3051.22
           799               116.71
           1703             5105.36
           1726             2671.70
           2119            19101.81
           2202              152.93
           3203             2430.48
           3504             5834.03
           3509            15023.06
           4401             1789.37
           4517             2993.39
           5122             3502.56
           5558             9622.52
           5616            10918.39
           5619             2092.68
           5824            14359.36
           6202             2082.01
           7205             9619.74
           9115             7191.49
           9909            30544.50
2012-03-03 706              3339.70
           799               222.00
           1703             6334.65
           1726             3697.27
           2119            25581.46
           2202              282.40
           3203             2913.40
           3504             6929.11
           3509            19179.72
           4401             1845.76
           4517             4244.25
           5122             4401.96
           5558            11082.70
           5616            15106.99
           5619             2783.77
           5824            19778.65
           6202             3014.61
           7205            12486.74
           9115             8503.97
           9909            39338.87
2012-03-04 706              3485.09
           799               190.15
           1703             6117.33
           1726             3866.30
           2119            23910.60
           2202              230.19
           3203             2993.55
           3504             6809.36
           3509            24502.11
           4401             1931.14
           4517             3874.49
           5122             4704.78
           5558            11445.21
           5616            16209.39
           5619             2753.26
           5824            19967.36
           6202             4025.54
           7205            13561.09
           9115             4157.50
           9909            50807.80
2012-03-05 706              2525.48
           799                99.86
           1703             4195.29
           1726             2501.84
           2119            14496.61
           2202              171.83
           3203             1970.25
           3504             4848.78
           3509            14383.39
           4401             1171.30
           4517             1998.86
           5122             3337.60
           5558             7661.99
           5616            10668.84
           5619             1710.66
           5824            12603.33
           6202             2692.12
           7205             8512.69
           9115             3481.45
           9909            33779.53
2012-03-06 706              2629.16
           799               128.55
           1703             3818.83
           1726             2121.07
           2119            14046.25
           2202              111.38
           3203             1857.88
           3504             4505.58
           3509            13688.41
           4401             1423.49
           4517             2190.48
           5122             3299.15
           5558             7628.23
           5616             8908.48
           5619             1546.11
           5824            11146.89
           6202             2542.13
           7205             7812.47
           9115             3388.68
           9909            28392.33
2012-03-07 706              2428.49
           799                57.63
           1703             3936.35
           1726             2110.38
           2119            14166.80
           2202              136.02
           3203             1867.16
           3504             4614.33
           3509            14398.19
           4401          

In [12]:
df2=aggregated/df
df2.sort_index()

purchaseamount  id  chain  dept  company  brand  productsize  productmeasure  purchasequantity
date       category                                                                                                
2012-03-02 706            10.557855 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           799             9.725833 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           1703            7.619940 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           1726            8.535783 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           2119            8.951176 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           2202            6.372083 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           3203            6.604565 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           3504            5.799235 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           3509           11.801304 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           4401            9.037222 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           4517           20.502671 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5122           10.711193 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5558            8.345637 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5616            7.866275 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5619            9.555616 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5824           10.271359 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           6202           10.515202 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           7205            6.299764 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           9115           70.504804 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           9909            5.820217 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
2012-03-03 706            10.949836 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           799            13.875000 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           1703            7.577333 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           1726            8.782114 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           2119           10.051654 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           2202            8.557576 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           3203            6.855059 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           3504            5.750299 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           3509           13.038559 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           4401            9.047843 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           4517           21.011139 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5122           12.093297 NaN    NaN   NaN      NaN    NaN          NaN             NaN               NaN
           5558            8.685502 NaN    NaN   NaN      NaN    NaN         

In [13]:
df3=df2.drop(columns=['id', 'chain','dept','company','brand','productsize','productmeasure','purchasequantity']).sort_index()
df3

purchaseamount
date       category                
2012-03-02 706            10.557855
           799             9.725833
           1703            7.619940
           1726            8.535783
           2119            8.951176
           2202            6.372083
           3203            6.604565
           3504            5.799235
           3509           11.801304
           4401            9.037222
           4517           20.502671
           5122           10.711193
           5558            8.345637
           5616            7.866275
           5619            9.555616
           5824           10.271359
           6202           10.515202
           7205            6.299764
           9115           70.504804
           9909            5.820217
2012-03-03 706            10.949836
           799            13.875000
           1703            7.577333
           1726            8.782114
           2119           10.051654
           2202            8.557576
           3203            6.855059
           3504            5.750299
           3509           13.038559
           4401            9.047843
           4517           21.011139
           5122           12.093297
           5558            8.685502
           5616            8.264218
           5619           10.706808
           5824            9.984175
           6202           10.431176
           7205            6.603247
           9115           71.461933
           9909            6.078317
2012-03-04 706            11.352085
           799            15.845833
           1703            7.883157
           1726            9.140189
           2119            9.507197
           2202           12.788333
           3203            6.977972
           3504            5.702982
           3509           12.868755
           4401            6.946547
           4517           19.180644
           5122           11.645495
           5558            8.948561
           5616            8.129082
           5619           10.548889
           5824           10.151174
           6202           11.567644
           7205            6.943722
           9115           59.392857
           9909            6.237147
2012-03-05 706            10.435868
           799             8.321667
           1703            7.373093
           1726            8.567945
           2119            8.552572
           2202            9.546111
           3203            6.481086
           3504            5.691056
           3509           12.117430
           4401            7.056024
           4517           18.680935
           5122           10.697436
           5558            8.355496
           5616            7.534492
           5619           10.826962
           5824            9.884965
           6202           10.434574
           7205            6.609231
           9115           65.687736
           9909            6.037450
2012-03-06 706            10.864298
           799             8.570000
           1703            7.458652
           1726            8.126705
           2119            8.873184
           2202            7.955714
           3203            6.131617
           3504            5.402374
           3509           11.340853
           4401            7.046980
           4517           16.980465
           5122           11.070973
           5558            8.193588
           5616            7.320033
           5619            9.723962
           5824           10.051298
           6202           12.105381
           7205            6.320769
           9115           59.450526
           9909            5.782552
2012-03-07 706            10.378162
           799            28.815000
           1703            6.798532
           1726            8.085747
           2119            8.622520
           2202            6.182727
           3203            6.350884
           3504            5.279554
           3509           11.257381
           4401          

In [14]:
#df3.to_csv(path='/data/season_test.csv',num_files=1)

In [15]:
df4=df3.fillna(value=0)

In [16]:
df4.toPandas().to_csv('./season_test.csv')

/usr/local/lib/python3.8/dist-packages/databricks/koalas/frame.py:4901: FutureWarning: DataFrame.toPandas is deprecated as of DataFrame.to_pandas. Please use the API instead.
  warnings.warn(


In [17]:
end = time.time()
print("執行時間：%f 秒" % (end - start))

執行時間：479.393210 秒
